### QuantLib Python – Twisting a Snake to fit a Yieldcurve

https://letyourmoneygrow.com/2018/04/14/quantlib-python-twisting-a-snake-to-fit-a-yieldcurve/

In [1]:
!pip install QuantLib

     |████████████████████████████████| 18.8MB 33.4MB/s 


In [2]:
import numpy as np
import QuantLib as ql

In [3]:
today = ql.Date(23, 03, 2021) # 일-월-년
ql.Settings.instance().evaluationDate = today

SyntaxError: ignored

In [ ]:
terminationDates = [ql.Date(4, 7, 2044), ql.Date(15, 2, 2028), ql.Date(14, 4, 2023)]
tenors = np.repeat(ql.Period(ql.Annual), 3) #allusion on R function rep()
calenders = np.repeat(ql.Germany(), 3)
termDateConvs = np.repeat(ql.Following, 3)
genRules = np.repeat(ql.DateGeneration.Backward, 3)
endOfMonths = np.repeat(False, 3)
firstDates = [ql.Date(27, 4, 2012), ql.Date(10, 1, 2018), ql.Date(2, 2, 2018)]

In [ ]:
settlementDays = np.repeat(2, 3)
coupons = [0.025, 0.005, 0.0]
cleanPrices = [126.18, 98.18, 99.73]
faceValues = np.repeat(100.0, 3)
dayCounts = np.repeat(ql.ActualActual(), 3)

In [ ]:
schedules = []
bonds = []
bondHelpers = []
for j in range(0, 3):
    # without int() and bool() conversion it will not work due to int vs. int32_ and bool vs bool_
    schedules.append(ql.Schedule(firstDates[j], terminationDates[j], tenors[j], calenders[j],
                                 int(termDateConvs[j]), int(termDateConvs[j]), int(genRules[j]),
                                 bool(endOfMonths[j])))
    bonds.append(ql.FixedRateBond(int(settlementDays[j]), float(faceValues[j]), schedules[j],
                                  [float(coupons[j])], dayCounts[j]))
    bondHelpers.append(ql.BondHelper(ql.QuoteHandle(ql.SimpleQuote(float(cleanPrices[j]))), bonds[j]))

In [ ]:
list(schedules[0])
list(schedules[1])
list(schedules[2])

[Date(2,2,2018),
 Date(16,4,2018),
 Date(15,4,2019),
 Date(14,4,2020),
 Date(14,4,2021),
 Date(14,4,2022),
 Date(14,4,2023)]

In [ ]:
print(bonds[0].bondYield(float(cleanPrices[0]), dayCounts[0], ql.Compounded, ql.Annual))
print(bonds[1].bondYield(float(cleanPrices[1]), dayCounts[1], ql.Compounded, ql.Annual))
print(bonds[2].bondYield(float(cleanPrices[2]), dayCounts[2], ql.Compounded, ql.Annual))

0.012142580037486018
0.007623722150325777
0.0011699550318717993


In [ ]:
curveSettlementDays = 2
curveCalendar = ql.Germany()
curveDaycounter = ql.ActualActual()

In [ ]:
#piecewise log cubic discount curve. Surprisingly there is no log-linear...
yieldCurve = ql.PiecewiseLogCubicDiscount(today, bondHelpers, curveDaycounter)
print(yieldCurve.discount(ql.Date(1, 1, 2021)))
print(yieldCurve.discount(ql.Date(1, 1, 2025)))
print(yieldCurve.discount(ql.Date(1, 1, 2035)))

0.9999999851565351
0.9854621222298319
0.8572908276833388


In [ ]:
##and Nelson-Siegel
curveFittingMethod = ql.NelsonSiegelFitting()
tolerance = 1.0e-5
iterations = 1000
yieldCurveNS = ql.FittedBondDiscountCurve(curveSettlementDays, curveCalendar, bondHelpers,
                                          curveDaycounter, curveFittingMethod, tolerance, iterations)
res = yieldCurveNS.fitResults()
print(yieldCurve.discount(ql.Date(1, 1, 2021)))
print(yieldCurve.discount(ql.Date(1, 1, 2025)))
print(yieldCurve.discount(ql.Date(1, 1, 2035)))

0.9999999851565351
0.9854621222298319
0.8572908276833388
